In [ ]:
import openai
from importlib import reload
from llm_client import llm_client
from llm_client.agent import MultiTurnAgent
from llm_client.prompt_builder import  PromptBuilder
from llm_client.document_vector_store import (
    EmbeddingProcessor,
    Document,
    DocumentStore,
    SimpleDocument,
    VectorStore
)
from pipelines.kme_doc import KMEDocument
from tqdm import tqdm
import pandas as pd
import os
from config.settings import Settings
from content_retrieval import query_document, llm,doc_store, summary_doc_store ,embed,embedding
settings = Settings()

ModuleNotFoundError: No module named 'implementations.kme_doc'

In [2]:
kme_vertaaltabel = pd.read_csv(settings.data_root /  "kme_vertaaltabel.csv",sep=';').set_index("KME_ID")

# Opbouwen Doc store van pdf

In [3]:
kme_documents = pd.read_pickle(settings.content_folder / "extracted_df_km.pkl").reset_index()
kme_documents = kme_documents.set_index("km_nummer").join(kme_vertaaltabel).reset_index()
ob_documents = kme_documents.query("BELASTINGSOORT == 'OB - Omzetbelasting'")
results = add_new_documents_to_docstore(ob_documents,doc_store)

In [4]:
summary_processor = PromptBuilder(template_path='prompt_templates',name='summarize')

### Samenvatten en segmenteren content

In [5]:
stats = summarize_new_documents(
    doc_store=doc_store,
    summary_doc_store=summary_doc_store,
    prompt_builder=summary_processor,
    llm=llm,
    KMEDocument_cls=KMEDocument,
    max_workers=10,
    start=0,
    count=1500,
    show_progress=True
)

Summarizing: 100%|██████████| 1/1 [00:04<00:00,  4.46s/it]


In [7]:
import json
if os.path.exists(settings.data_root / "project_list.json"):
    with open(settings.data_root / "project_list.json") as f:
        project_list = json.load(f)
        for project in project_list:
            project

In [9]:
project['question']

'OB - Welke gegevens heb ik ingevuld op mijn aangifte?'